In [59]:
from pymongo import MongoClient
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

In [60]:
import re
comp= {"name":{"$exists":True}}
cur = db.companies.find(comp)
comp2 = list(cur)

proyection ={"offices":1, "name":1}

comp_result= list(db.companies.find(comp,proyection))

In [61]:
import pandas as pd

df = pd.DataFrame(list(comp_result))
df


,_id,name,offices
0,52cdef7c4bab8bd675297d8b,AdventNet,"[{'description': 'Headquarters', 'address1': '..."
1,52cdef7c4bab8bd675297d8c,Zoho,"[{'description': 'Headquarters', 'address1': '..."
2,52cdef7c4bab8bd675297d8f,Omnidrive,"[{'description': '', 'address1': 'Suite 200', ..."
3,52cdef7c4bab8bd675297d8a,Wetpaint,"[{'description': '', 'address1': '710 - 2nd Av..."
4,52cdef7c4bab8bd675297d90,Postini,"[{'description': None, 'address1': '959 Skyway..."
...,...,...,...
18796,52cdef7f4bab8bd67529c6f8,goBookmaker,[]
18797,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,"[{'description': '', 'address1': '', 'address2..."
18798,52cdef7f4bab8bd67529c6fa,EnteGreat,"[{'description': '', 'address1': '1900 Interna..."
18799,52cdef7f4bab8bd67529c6f7,AfterLogic,"[{'description': 'Livingston', 'address1': 'P...."


In [62]:
offices = df.explode("offices").apply(lambda e: e.offices,axis=1,result_type="expand")
offices

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
3,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
3,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
...,...,...,...,...,...,...,...,...,...
18796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18797,,,,,Birmingham,AL,USA,33.518885,-86.816068
18798,,1900 International Park Drive,Suite 200,35243-4204,Birmingham,AL,USA,33.407786,-86.761302
18799,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554


In [63]:
offices.dtypes

description      object
address1         object
address2         object
zip_code         object
city             object
state_code       object
country_code     object
latitude        float64
longitude       float64
dtype: object

In [64]:
clean_offices = pd.concat([df[["name","_id"]], offices], axis=1)
clean_offices

,name,_id,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,AdventNet,52cdef7c4bab8bd675297d8b,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,Zoho,52cdef7c4bab8bd675297d8c,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,Omnidrive,52cdef7c4bab8bd675297d8f,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
3,Wetpaint,52cdef7c4bab8bd675297d8a,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
3,Wetpaint,52cdef7c4bab8bd675297d8a,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
...,...,...,...,...,...,...,...,...,...,...,...
18796,goBookmaker,52cdef7f4bab8bd67529c6f8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18797,EnteGreat Solutions,52cdef7f4bab8bd67529c6f9,,,,,Birmingham,AL,USA,33.518885,-86.816068
18798,EnteGreat,52cdef7f4bab8bd67529c6fa,,1900 International Park Drive,Suite 200,35243-4204,Birmingham,AL,USA,33.407786,-86.761302
18799,AfterLogic,52cdef7f4bab8bd67529c6f7,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554


In [65]:
import numpy as np

def transformToGeoPoint(s):
    if np.isnan(s.latitude) or np.isnan(s.longitude):
        return None
    return {
        "type":"Point",
        "coordinates":[s.longitude, s.latitude]
    }
    

clean_offices["geopoint"] = clean_offices.apply(transformToGeoPoint, axis=1)

In [66]:
companies = db.companies.find({},{"offices":1, "name":1})
df = pd.DataFrame(list(companies))
offices = df.explode("offices").apply(lambda e: e.offices,axis=1,result_type="expand")
clean_offices = pd.concat([df[["name","_id"]], offices], axis=1)
clean_offices["geopoint"] = clean_offices.apply(transformToGeoPoint, axis=1)

In [67]:
clean_offices = clean_offices.rename(columns={"_id":"company_id"})
clean_offices.head()

,name,company_id,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,geopoint
0,AdventNet,52cdef7c4bab8bd675297d8b,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."
1,Zoho,52cdef7c4bab8bd675297d8c,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."
2,Omnidrive,52cdef7c4bab8bd675297d8f,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN,None
3,Wetpaint,52cdef7c4bab8bd675297d8a,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
3,Wetpaint,52cdef7c4bab8bd675297d8a,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."


In [68]:
def c_data(df,column):
     print(df[f"{column}"].value_counts())

In [69]:
c_data(clean_offices,"country_code")

USA    10229
GBR     1385
CAN      621
IND      519
DEU      407
       ...  
IRQ        1
PST        1
ARM        1
MTQ        1
BOL        1
Name: country_code, Length: 114, dtype: int64


In [70]:
usa2.isnull().sum().sort_values(ascending=False)

geopoint        0
longitude       0
latitude        0
country_code    0
state_code      0
city            0
zip_code        0
address2        0
address1        0
description     0
company_id      0
name            0
dtype: int64

In [71]:
drop_unresponsive= ['USA']
usa2 = clean_offices[(clean_offices["country_code"].isin(drop_unresponsive))]

In [72]:
def drop_null(table,subset):
    table.dropna(subset=[f"{subset}"],inplace=True)

In [73]:
list(usa2.columns)
for i in list(usa2.columns):
    drop_null(usa2,f"{i}")

<ipython-input-72-58feeac4b584>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table.dropna(subset=[f"{subset}"],inplace=True)


In [74]:
usa2.isnull().sum().sort_values(ascending=False)

geopoint        0
longitude       0
latitude        0
country_code    0
state_code      0
city            0
zip_code        0
address2        0
address1        0
description     0
company_id      0
name            0
dtype: int64

In [75]:
usa2["description"].value_counts()

                            2344
Headquarters                 798
HQ                           527
Corporate Headquarters       351
World Headquarters           130
                            ... 
ZIPCodeWorld.com               1
Business Operations            1
Infochimps House of Data       1
Manhattan                      1
Aprius Education               1
Name: description, Length: 1256, dtype: int64

In [76]:

clean_offices["company_id"] = clean_offices["company_id"].apply(lambda e: str(e))

In [77]:
clean_offices.to_json("../input/offices.json",orient="records")

In [82]:
from haversine import haversine, Unit

lyon = (45.7597, 4.8422) # (lat, lon)
paris = (48.8567, 2.3508)

In [83]:
haversine(lyon, paris)

392.2172595594006